In [1]:
from neural_features import ResNet50FeatureExtractor
import numpy as np
import pandas as pd
from torchvision.models import resnet50
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import shutil
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from neighbor_distances import CentroidWeights
from sklearn.model_selection import train_test_split

In [2]:
def image_retrieval_using_knn(n_neighbors, dataset, queryset):
    pipeline = Pipeline([
        ('knn', KNeighborsClassifier(n_neighbors=n_neighbors))  
    ])

    pipeline.fit(dataset[0], dataset[1])
    predictions = pipeline.predict(queryset)

    return predictions
    

In [3]:
train_small_names = pd.read_csv('../dataset/train_small.csv', header=None)
train_unlabeled_names = pd.read_csv('../dataset/train_small.csv', header=None)

small_dataset = np.load('../dataset/small_train_features_resnet50.npz', 'r')
X = small_dataset['X']
X_label = small_dataset['y']

queryset = np.load('../dataset/train_unlabeled_features_resnet50_completed.npz', 'r')
Q = queryset['X']
Q_label = queryset['y']

In [5]:
int(len(Q) * 0.20)

22691

In [6]:
extracted_indeces = np.random.choice(Q.shape[0], size=int(len(Q) * 0.20), replace=False)
print(extracted_indeces)


[ 84024  49532  84620 ...  17969 103079  15921]


In [7]:
Q_small = Q[extracted_indeces, :]
Q_small.shape

(22691, 2048)

In [8]:
predictions = image_retrieval_using_knn(5, (X, X_label), Q_small)


In [13]:
test_size = int((.2 * (Q_small.shape[0] + X.shape[0]) )/ 251)/20
if test_size >= 1:
    test_size = 1.0


In [22]:
if test_size != 1:
    X_train, X_test, y_train, y_test = train_test_split(X, X_label, test_size=test_size, stratify=X_label, random_state=42)
else:
    X_train, X_test, y_train, y_test = [], X, [], X_label

print(X_train, X_test.shape, y_train, y_test.shape)

[] (5020, 2048) [] (5020,)


In [23]:
if X_train == []:
    X_train = Q_small
    y_train = predictions # Combine
else:
    X_train = np.concatenate((X_train, Q_small), axis=0)  # Combine rows
    y_train = np.concatenate((y_train, predictions), axis=0) 

In [24]:
X_train.shape

(22691, 2048)

In [25]:
y_train.shape

(22691,)

In [26]:
X_test.shape

(5020, 2048)

In [27]:
y_test.shape

(5020,)

In [28]:
np.savez("../dataset/features_extended_20.npz", X_train = X_train, y_train = y_train, X_val=X_test, y_val=y_test)